## 準備
RAGエージェントを作成するには次の3つの要素が必要となる。

- model
- embedding
- vector store


### model

In [ ]:
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
import os

load_dotenv()

GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]


In [ ]:
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY

model = init_chat_model("google_genai:gemini-2.5-flash-lite")




### embedding

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")


### vector store

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

## Indexing

Indexingは次の流れとなっている。
- 文書の読み込み
- 文書の分割
- Store

### 文書の読み込み

In [ ]:
from langchain_community.document_loaders import WebBaseLoader


loader = WebBaseLoader(
    web_paths=("https://example.com/","https://github.com/arkyork/LangGraph-learn"),
)

In [ ]:
documents = loader.load()

### 文書の分割

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 500,
    add_start_index = True
)

In [ ]:
data_split = text_splitter.split_documents(documents)


### Store

In [ ]:
ids = vector_store.add_documents(documents=data_split)


In [ ]:
ids

## Retrieval と Generation

In [ ]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrive(query):
    """情報の検索"""
    relventa_docs = vector_store.similarity_search(query,k=3)
    serialized = "\n\n".join(
        (f"Metadata: {doc.metadata}\nContent: {doc.page_content}")
        for doc in relventa_docs
    )
    return serialized, relventa_docs

In [ ]:
from langchain.agents import create_agent

tools = [retrive]

# System Prompt
system_prompt = (
    "あなたは記事から情報を取得できるツールにアクセスできます。 "
    "そのツールを使ってユーザーからの質問に回答してください。"
    "回答は日本語で行ってください。"
)

agent = create_agent(model,tools,system_prompt=system_prompt)

In [ ]:
prompt = (
 "あなたのブログ内でLangGraphについて説明している記事を検索して要約して"
)


In [ ]:
for event in agent.stream(
    {"messages":[{"role":"user","content":prompt}]},
    stream_mode="values"
):
    event["messages"][-1].pretty_print()

